<a href="https://colab.research.google.com/github/Dohy-Lee/NLP_by_Pytorch/blob/main/ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 샘플(데이터 포인트) : 메타데이터가 붙은 텍스트
# 데이터셋 : 샘플의 모음인 말뭉치
# 토큰화 : 텍스트를 토큰으로 나누는 과정. 보통 구두점이나 공백 문자로 구분.
# 한국어와 튀르키예어 같은 교착어(어근과 접사가 단어의 기능을 결정하는 언어)는 공백 문자와 구두점으로 나누는 것으론 부족함.
# 텍스트를 바이트 스트림으로 신경망에 표현하여 이와 같은 문제를 완전히 피해 갈 수 있음

## 텍스트 처리 분야에 널리 쓰이는 패키지 NLTK을 통해 텍스트 토큰화

In [11]:
!python -m spacy download en
import spacy
nlp = spacy.load('en_core_web_sm')
text = "Mary, don't slap the green witch."
print([str(token) for token in nlp(text.lower())])

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 3.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
['mary', ',', 'do', "n't", 'slap', 'the', 'green', 'witch', '.']


In [13]:
from nltk.tokenize import TweetTokenizer
tweet=u"Snow White and the Seven Degrees #MakeAMovieCold@midnight:-)"
tokenizer = TweetTokenizer()
print(tokenizer.tokenize(tweet.lower()))

['snow', 'white', 'and', 'the', 'seven', 'degrees', '#makeamoviecold', '@midnight', ':-)']


In [ ]:
# 타입 : 말뭉치에 등장하는 고유한 토큰. 말뭉치에 있는 모든 타입의 집합이 어휘 사전 또는 어휘임
# 단어는 내용어와 불용어로 구분됨. 관사와 전치사 같은 불용어는 대부분 내용어를 보충하는 문법적인 용도로 사용

## 유니그램, 바이그램, 트라이그램, ⋯, n-그램
* n-그램 : 텍스트에 있는 고정 길이(n)의 연속된 토큰 시퀀스
* 바이그램 : 토큰 두 개
* 유니그램 : 토큰 한개

In [15]:
# 텍스트에서 n-그램 만들기
def n_grams(text,n):
  '''
  takes tokens or text, returns a list of n-grams
  '''
  return [text[i:i+n] for i in range(len(text)-n+1)]

cleand = ['mary', ',', "n't", 'slap', 'green', 'witch', '.']
print(n_grams(cleand,3))

[['mary', ',', "n't"], [',', "n't", 'slap'], ["n't", 'slap', 'green'], ['slap', 'green', 'witch'], ['green', 'witch', '.']]


## 표제어와 어간
* 표제어 : 단어의 기본형, 즉 사전에 등재된 단어
* 표제어 추출 : 토큰을 표제어로 바꾸어 벡터 표현의 차원을 줄이는 방법
* 어간 추출 : 표제어 추출 대신 사용하는 축소 기법. 수동으로 만든 규칙을 사용해 단어의 끝을 잘라 어간이라는 공통 형태로 축소

 ex) geese를 표제어 추출할 경우 goose, 어간 추출할 경우 gees가 생성

In [16]:
# 해당 단어를 표제어로 변환
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"he was running late")
for token in doc:
  print('{} --> {}'.format(token, token.lemma_))

he --> he
was --> be
running --> run
late --> late


## 문장과 문서 분류하기
* 문서를 분류하는 작업은 NLP 분야의 초기 애플리케이션 중 하나
* TF와 TF-IDF 표현이 문서나 문장 같은 긴 텍스트 뭉치를 분류하는 데 유용

## 단어 분류하기: 품사 태깅


In [17]:
# 품사 태깅
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"Mary slapped the green witch.")
for token in doc:
  print('{} - {}'.format(token, token.pos_))

Mary - PROPN
slapped - VERB
the - DET
green - ADJ
witch - NOUN
. - PUNCT


## 청크 나누기와 개체명 인식
* 청크나누기(부분 구문 분석) : 연속된 여러 토큰으로 구분되는 텍스트 구에 레이블을 할당
 * 명사, 동사, 형용사 같은 문법 요소로 구성된 고차원의 단위를 유도해 내는 것이 목표
 * 부분 구문 분석 모델 훈련에 사용할 데이터가 없다면 품사 태깅에 정규식을 적용해 부분 구문 분석을 근사할 수 있음
* 개채명 : 사람, 장소, 회사, 약 이름 등 실제 세상의 개념을 의미하는 문자열

In [18]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Mary slapped the green witch.')
for chunk in doc.noun_chunks:
  print('{} - {}'.format(chunk, chunk.label_))

Mary - NP
the green witch - NP
